# Azure AI Document Intelligenceを使用して複雑なPDFからQnA合成データセットを生成する

### 概要
PDFを3つの部分に分けて処理します。

- **混合ページ（画像とテキストが適切に混在）** - Azure AI Document Intelligenceでドキュメントを読み取った後、図のタグ内の画像説明をマルチモーダルLLMで要約されたテキストに置き換えます。（しばしば画像説明は空白か短いキャプションのみです。）
- **Text-heavy** - テキストが多いPDFは、Azure AI Document IntelligenceやUnstructuredのようなツールキットを使用せずに、オープンソースで処理できます。
- **Image-heavy** - 画像が多いPDFは、ページ全体を画像に変換し、GPT-4oのようなマルチモーダルLLMに各ページを要約させます。

In [ ]:
! pip install -r requirements.txt

## 0. 事前準備
---

- Azure Portal から Document intelligence を作成し、エンドポイント・キーをコピー。
- Document intelligence のエンドポイント・キー、Azure OpenAI のエンドポイント・キーを環境変数にセット。

In [ ]:
import os


os.environ["AZURE_DOC_INTELLIGENCE_ENDPOINT"] = "https://xxxxxxx.cognitiveservices.azure.com/"
os.environ["AZURE_DOC_INTELLIGENCE_KEY"] = "xxxxxxx"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://oai-xxxxxxx.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "xxxxxxx"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-4o"


## 1. Read & Preprocess PDF file
---

### PDFを個々のページに分割する
テストのためにPDFドキュメントの一部のみを使用します

In [ ]:
import openai
from dotenv import load_dotenv
from util.common_utils import get_language_code

load_dotenv()

raw_data_dir = "../contents"
splitted_raw_data_dir = "splitted_contents"
file_path = f"{raw_data_dir}/azure-ai-search-overview.pdf"

DOMAIN = "Distributed training on Cloud"
LANGUAGE = "Japanese" # You can change your language here. e.g., "Korean", "English", "Chinese"
LANGUAGE_CODE = get_language_code(LANGUAGE)
print(f"Domain: {DOMAIN}, Language: {LANGUAGE}, Language Code: {LANGUAGE_CODE}")

テストのためにPDFドキュメントの一部のみを使用します。ページが多い場合や部分的な処理が必要な場合は、一部のページのみを切り取って保存します。

In [ ]:
import fitz

# Open the first PDF document
doc1 = fitz.open(file_path)
split_pages = [(5, 25)]

for idx, s in enumerate(split_pages):
    # Create a new empty PDF document
    doc2 = fitz.open()

    # Insert the first 2 pages of doc1 into doc2
    doc2.insert_pdf(doc1, from_page=s[0], to_page=s[1])

    # Save the modified document
    doc2.save(f"{raw_data_dir}/part{idx}.pdf")

ファイルエクスプローラーから、作成された `contents/part0.pdf` を確認してみます。PDF の一部が切り取られていることがわかります。

![part0.pdf](../images/appendix-01.png)

主にテキストで構成されたページ、主に画像で構成されたページ、およびテキストと画像が混在するページを区別します。

In [ ]:
from util.common_utils import delete_folder_and_make_folder
from util.preprocess import analyze_pdf_page_content, split_pdf

file_path = f"{raw_data_dir}/part0.pdf"
analyzed_pdf_result = analyze_pdf_page_content(file_path)
delete_folder_and_make_folder(splitted_raw_data_dir)    

print("### PDF Content Analysis Result:")
for content_type, pages in analyzed_pdf_result.items():
    print(f"{content_type} pages: {pages}")
    split_pdf(file_path, f"{splitted_raw_data_dir}/{content_type}.pdf", pages)

In [ ]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import ContentFormat
from openai import AzureOpenAI

doc_intelligence_endpoint = os.getenv("AZURE_DOC_INTELLIGENCE_ENDPOINT")
doc_intelligence_key = os.getenv("AZURE_DOC_INTELLIGENCE_KEY")

document_intelligence_client = DocumentIntelligenceClient(
    endpoint=doc_intelligence_endpoint, 
    credential=AzureKeyCredential(doc_intelligence_key),
    headers={"x-ms-useragent":"sample-code-figure-understanding/1.0.0"},
)

aoai_api_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
aoai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
aoai_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

client = AzureOpenAI(
    api_key=aoai_api_key,  
    api_version=aoai_api_version,
    base_url=f"{aoai_api_endpoint}/openai/deployments/{aoai_deployment_name}",
    max_retries=1
)

### ケース1: 混合ページ（画像とテキストが適切に混在）の場合
Azure AI Document Intelligenceでドキュメントを読み取った後、`img`タグ内の画像説明をマルチモーダルLLMで要約されたテキストに置き換えます。（しばしば画像説明は空白か短いキャプションのみです。）

#### ドキュメントの分析

In [ ]:
if "Mixed" in analyzed_pdf_result:
    pdf_mixed_path = f"{splitted_raw_data_dir}/Mixed.pdf"

    with open(pdf_mixed_path, "rb") as f:
        poller = document_intelligence_client.begin_analyze_document(
            "prebuilt-layout", analyze_request=f, content_type="application/octet-stream", 
            output_content_format=ContentFormat.MARKDOWN 
        )

    result = poller.result()
    md_content = result.content

    #### Updates the content of the figure description (empty content or caption) with the image summary text generated by gpt-4o.
    from util.preprocess import (
        image_complexity, is_bounding_box_larger_than, crop_image_from_file, 
        understand_image_with_gpt, update_figure_description
    )
    output_folder = "pdf_mixed_tmp"
    delete_folder_and_make_folder(output_folder)
    language = LANGUAGE
    max_tokens = 1024
    input_file_path = file_path

    if result.figures:
        print("Figures:")
        for idx, figure in enumerate(result.figures):
            figure_content = ""
            img_description = ""
            
            for i, span in enumerate(figure.spans):
                figure_content += md_content[span.offset:span.offset + span.length]

            # Note: figure bounding regions currently contain both the bounding region of figure caption and figure body
            if figure.caption:
                caption_region = figure.caption.bounding_regions
                for region in figure.bounding_regions:
                    if region not in caption_region:
                        boundingbox = (
                                region.polygon[0],  # x0 (left)
                                region.polygon[1],  # y0 (top)
                                region.polygon[4],  # x1 (right)
                                region.polygon[5]   # y1 (bottom)
                            )

                        if is_bounding_box_larger_than(boundingbox):
                            cropped_image = crop_image_from_file(pdf_mixed_path, region.page_number - 1, boundingbox) # page_number is 1-indexed

                            if image_complexity(cropped_image)[0] == "Complex":
                                # Get the base name of the file
                                base_name = os.path.basename(input_file_path)
                                # Remove the file extension
                                file_name_without_extension = os.path.splitext(base_name)[0]

                                output_file = f"{file_name_without_extension}_cropped_image_{idx}.png"
                                cropped_image_filename = os.path.join(output_folder, output_file)

                                cropped_image.save(cropped_image_filename)
                                print(f"\tFigure {idx} cropped and saved as {cropped_image_filename}")

                                try: 
                                    image_summarization = understand_image_with_gpt(client, aoai_deployment_name, cropped_image_filename, "", max_tokens=max_tokens, language=language)
                                except openai.BadRequestError as e:
                                    print(f"BadRequestError: {e}")
                                    image_summarization = ""
                                img_description += image_summarization

                                print(f"\tDescription of figure {idx}: {img_description}")
                            else:
                                print(f'simple image at idx {idx}')

            else:
                for region in figure.bounding_regions:

                    # To learn more about bounding regions, see https://aka.ms/bounding-region
                    boundingbox = (
                            region.polygon[0],  # x0 (left)
                            region.polygon[1],  # y0 (top
                            region.polygon[4],  # x1 (right)
                            region.polygon[5]   # y1 (bottom)
                        )

                    if is_bounding_box_larger_than(boundingbox):                    

                        cropped_image = crop_image_from_file(input_file_path, region.page_number - 1, boundingbox) # page_number is 1-indexed

                        if image_complexity(cropped_image)[0] == "Complex":
                            # Get the base name of the file
                            base_name = os.path.basename(input_file_path)
                            # Remove the file extension
                            file_name_without_extension = os.path.splitext(base_name)[0]

                            output_file = f"{file_name_without_extension}_cropped_image_{idx}.png"
                            cropped_image_filename = os.path.join(output_folder, output_file)
                            # cropped_image_filename = f"data/cropped/image_{idx}.png"
                            cropped_image.save(cropped_image_filename)

                            try:
                                image_summarization = understand_image_with_gpt(client, aoai_deployment_name, cropped_image_filename, "", max_tokens=max_tokens, language=language)
                            except openai.BadRequestError as e:
                                print(f"BadRequestError: {e}")
                                image_summarization = ""
                            img_description += image_summarization
                            print(f"\tDescription of figure {idx}: {img_description}")
                        else:
                            print(f'simple image at idx {idx}')

            
            md_content = update_figure_description(md_content, img_description, idx)    

#### 混合ページのチャンクを生成

In [ ]:
if "Mixed" in analyzed_pdf_result:
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    import re

    text_splitter = RecursiveCharacterTextSplitter(
        separators=[
            r'<!-- PageNumber="\d+" -->',
            r"\n\n",
            r"\n",
            " ",
            ".",
            "",
        ],   
        is_separator_regex = True,    
        chunk_size=2000,
        chunk_overlap=200,
    )

    mixed_chunks = text_splitter.split_text(md_content)
    print("Length of splits (mixed case): " + str(len(mixed_chunks)))
else:
    mixed_chunks = []

### ケース2: テキストが多い場合
テキストが多いPDFは、Azure AI Document IntelligenceやUnstructuredのようなツールキットを使用せずに、オープンソースで処理できます。

In [ ]:
if "Text" in analyzed_pdf_result:
    from langchain_community.document_loaders.pdf import PyMuPDFLoader
    from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

    pdf_text_path = f"{splitted_raw_data_dir}/Text.pdf"
    loader = PyMuPDFLoader(pdf_text_path)
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1200, 
        chunk_overlap=200
    )

    text_chunks = text_splitter.split_documents(documents)

    for idx, chunk in enumerate(text_chunks):
        print(f"Chunk {idx}\n{chunk}")
        print("="*80)
        if idx == 2:
            break

    text_chunks = [d.page_content for d in text_chunks]
    print("Length of splits (text-heay case): " + str(len(text_chunks)))
else:
    text_chunks = []

### ケース3: 画像が多い場合
画像が多いPDFは、ページ全体を画像に変換し、GPT-4oのようなマルチモーダルLLMに各ページを要約させます。

#### 画像の前処理

In [ ]:
if "Image" in analyzed_pdf_result:
    import fitz
    from glob import glob

    image_dir = "./pdf_image_tmp"
    delete_folder_and_make_folder(image_dir) 

    pdf_image_path = f"{splitted_raw_data_dir}/Image.pdf"
    doc = fitz.open(pdf_image_path)
    clip_x, clip_y = 10, 10

    for i, page in enumerate(doc):
        x, y, w, h = page.rect
        clip = fitz.Rect(x+clip_x, y+clip_y, w-clip_x, h-clip_y)
        page.set_cropbox(clip)
        pix = page.get_pixmap()
        pix.save(f"{image_dir}/page_{i:03d}.jpg")

    images = sorted(glob(os.path.join(image_dir, "*.jpg")))

In [ ]:
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_openai import AzureChatOpenAI

max_tokens = 1024
llm = AzureChatOpenAI(
    temperature=0, 
    max_tokens=max_tokens,
    openai_api_version=aoai_api_version,
    azure_deployment=aoai_deployment_name                       
)

human_prompt_main = f"Given image, give a concise summary in {LANGUAGE}. Don't insert any XML tag such as <text> and </text> when answering."

system_prompt = "You are an assistant tasked with describing table or image, specialized in Smartphone product."
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)
human_prompt = [
    {
        "type": "image_url",
        "image_url": {
            "url": "data:image/png;base64," + "{image_base64}",
        },
    },
    {
        "type": "text",
        "text": human_prompt_main
    },
]
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

prompt = ChatPromptTemplate.from_messages(
    [
        system_message_template,
        human_message_template
    ]
)

summarize_chain = prompt | llm | StrOutputParser()

In [ ]:
%%time
if "Image" in analyzed_pdf_result:
    from util.preprocess import encode_image_base64
    #images = glob(os.path.join(image_path, "*.jpg"))
    base64_images = [encode_image_base64(img_path) for img_path in images]
    image_summaries = summarize_chain.batch(base64_images, {"max_concurrency": 8})
    image_summaries = remove_short_sentences(image_summaries)
    print("Length of image_summaries (image-heavy case): " + str(len(image_summaries)))
else:
    image_summaries = []

## 2. Q&Aペアの構築
----

### オプション1.
azure-ai-generativeパッケージを活用します。このパッケージのQADataGeneratorクラスを使用すると、QnAの合成質問を簡単に生成できます。ただし、このクラスをそのまま使用するとカスタムプロンプトを使用できないという欠点があるため、これを継承してCustomQADataGeneratorクラスを作成しました。

In [ ]:
from util.qa import CustomQADataGenerator
model_config = {
    "deployment": os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    "model": "gpt-4o",
    "max_tokens": 2000,
}

qa_generator = CustomQADataGenerator(model_config=model_config, templates_dir=f"./prompt_template/{LANGUAGE_CODE}")

In [ ]:
import asyncio
from collections import Counter
from typing import Dict
import os
from azure.ai.generative.synthetic.qa import QAType
concurrency = 6  # number of concurrent calls
sem = asyncio.Semaphore(concurrency)

#qa_type = QAType.CONVERSATION
qa_type = QAType.LONG_ANSWER

async def generate_async(text: str) -> Dict:
    async with sem:
        return await qa_generator.generate_async(
            text=text,
            qa_type=qa_type,
            num_questions=50,  # Number of questions to generate per text
        )

In [ ]:
input_batch = mixed_chunks + text_chunks + image_summaries
results = await asyncio.gather(*[generate_async(text) for text in input_batch], return_exceptions=True)

question_answer_list = []
token_usage = Counter()
for result in results:
    if isinstance(result, Exception):
        raise result  # exception raised inside generate_async()
    question_answer_list.append(result["question_answers"])
    token_usage += result["token_usage"]

print("Successfully generated QAs")

In [ ]:
question_answer_list

## 3. jsonl形式で保存
---

In [ ]:
import json
from util.common_utils import convert_to_oai_format, convert_to_jsonl_format, save_jsonl

output_dir = './dataset'
os.makedirs(output_dir, exist_ok=True)

system_prompt_msg = f"""You are the SME (Subject Matter Expert) in {DOMAIN}. Please answer the questions accurately. If the question is in {LANGUAGE}, write your answer in {LANGUAGE}."""

save_filename = "imagenet-training-summary"
oai_qa_pair = convert_to_oai_format(question_answer_list, system_prompt_msg=system_prompt_msg)

save_jsonl(oai_qa_pair, f"{output_dir}/{save_filename}-oai.jsonl")

## 4. AI Foundry での評価用にjsonl形式で保存
---

In [ ]:
for qa in question_answer_list:
    qa_pair = convert_to_jsonl_format(qa)
    print(qa_pair)
    save_jsonl(qa_pair, f"{output_dir}/{save_filename}-eval.jsonl")

## クリーンアップ

In [ ]:
!rm -rf {splitted_raw_data_dir} pdf_image_tmp pdf_mixed_tmp outputs_tmp images {output_dir}